In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn import model_selection, datasets
from sklearn.tree import DecisionTreeClassifier
import joblib
import pickle

In [3]:
mapping = {
  'B-OtherPER':'B-Person', 'B-SportsManager':'B-Person', 'B-Cleric':'B-Person', 'B-Politician':'B-Person', 'B-Athlete':'B-Person', 'B-Artist':'B-Person', 'B-Scientist':'B-Person',
  'B-ORG':'B-Group', 'B-TechCorp':'B-Group', 'B-CarManufacturer':'B-Group', 'B-SportsGRP':'B-Group', 'B-AerospaceManufacturer':'B-Group', 'B-OtherCorp':'B-Group', 'B-PrivateCorp':'B-Group', 'B-PublicCorp':'B-Group', 'B-MusicalGRP':'B-Group',
  'B-VisualWork':'B-CreativeWorks', 'B-MusicalWork':'B-CreativeWorks', 'B-WrittenWork':'B-CreativeWorks', 'B-ArtWork':'B-CreativeWorks', 'B-Software':'B-CreativeWorks', 'B-OtherCW':'B-CreativeWorks',
  'B-Facility':'B-Location', 'B-OtherLOC':'B-Location', 'B-HumanSettlement':'B-Location', 'B-Station':'B-Location',
  'B-Disease':'B-Medical', 'B-Symptom':'B-Medical', 'B-AnatomicalStructure':'B-Medical', 'B-MedicalProcedure':'B-Medical', 'B-Medication/Vaccine':'B-Medical', 
  'B-OtherPROD':'B-Product', 'B-Drink':'B-Product', 'B-Food':'B-Product', 'B-Vehicle':'B-Product', 'B-Clothing':'B-Product',

  'I-OtherPER':'I-Person', 'I-SportsManager':'I-Person', 'I-Cleric':'I-Person', 'I-Politician':'I-Person', 'I-Athlete':'I-Person', 'I-Artist':'I-Person', 'I-Scientist':'I-Person',
  'I-ORG':'I-Group', 'I-TechCorp':'I-Group', 'I-CarManufacturer':'I-Group', 'I-SportsGRP':'I-Group', 'I-AerospaceManufacturer':'I-Group', 'I-OtherCorp':'I-Group', 'I-PrivateCorp':'I-Group', 'I-PublicCorp':'I-Group', 'I-MusicalGRP':'I-Group',
  'I-VisualWork':'I-CreativeWorks', 'I-MusicalWork':'I-CreativeWorks', 'I-WrittenWork':'I-CreativeWorks', 'I-ArtWork':'I-CreativeWorks', 'I-Software':'I-CreativeWorks', 'I-OtherCW':'I-CreativeWorks',
  'I-Facility':'I-Location', 'I-OtherLOC':'I-Location', 'I-HumanSettlement':'I-Location', 'I-Station':'I-Location',
  'I-Disease':'I-Medical', 'I-Symptom':'I-Medical', 'I-AnatomicalStructure':'I-Medical', 'I-MedicalProcedure':'I-Medical', 'I-Medication/Vaccine':'I-Medical', 
  'I-OtherPROD':'I-Product', 'I-Drink':'I-Product', 'I-Food':'I-Product', 'I-Vehicle':'I-Product', 'I-Clothing':'I-Product',

  '_ O':'_ O', 'O':'O'
}

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

# Download and read in the data
train_data_file = "/content/drive/MyDrive/Colab Notebooks/DL_ASS2/EN-English/en_train.conll"
val_data_file = "/content/drive/MyDrive/Colab Notebooks/DL_ASS2/EN-English/en_dev.conll"
test_data_file = "/content/drive/MyDrive/Colab Notebooks/DL_ASS2/EN-English/en_test.conll"
def read_data(file_path):
    dataset = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lst = []
        for line in f:
            if line == '\n' or line[0] == '#':
                if lst:
                  dataset.append(lst)
                  lst = []
            else:
                parts = line.strip().split(' _ _ ', 1)
                word = parts[0]
                tag = parts[1] 
                tag = mapping[tag]
                lst.append((word, tag))
    return dataset

train_dataset = read_data(train_data_file)
print("Train")
for i in range(5):
  print(train_dataset[i])
dev_dataset = read_data(val_data_file)
print("Dev")
for i in range(5):
  print(dev_dataset[i])
test_dataset = read_data(test_data_file)
print("Test")
for i in range(5):
  print(test_dataset[i])
dataset =train_dataset + dev_dataset + test_dataset
print(len(train_dataset), len(dev_dataset), len(test_dataset), len(dataset)) 
print(len(train_dataset)+len(dev_dataset)+len(test_dataset))

# SEQ_LEN = min(100, (len(max(train_dataset, key=len))))
# print(len(max(dev_dataset, key=len)))
# print(len(max(test_dataset, key=len)))

Train
[('robert', 'B-Person'), ('gottschalk', 'I-Person'), ('1939', 'O'), ('academy', 'B-CreativeWorks'), ('award', 'I-CreativeWorks'), ('winner', 'O'), ('and', 'O'), ('founder', 'O'), ('of', 'O'), ('panavision', 'B-Group')]
[('during', 'O'), ('the', 'O'), ('reign', 'O'), ('of', 'O'), ('the', 'O'), ('tongzhi', 'B-Person'), ('emperor', 'I-Person'), ('(', 'O'), ('r', 'O'), ('.', 'O'), ('1861', 'O'), ('–', 'O'), ('1875', 'O'), (')', 'O'), (':', 'O')]
[('further', 'O'), ('research', 'O'), ('led', 'O'), ('in', 'O'), ('the', 'O'), ('1960s', 'O'), ('to', 'O'), ('the', 'O'), ('bahadur', 'B-Person'), ('representation', 'O'), ('which', 'O'), ('provides', 'O'), ('information', 'O'), ('about', 'O'), ('the', 'O'), ('errorbounds', 'O'), ('.', 'O')]
[('the', 'O'), ('ideas', 'O'), ('were', 'O'), ('introduced', 'O'), ('by', 'O'), ('william', 'B-Person'), ('burnside', 'I-Person'), ('at', 'O'), ('the', 'O'), ('end', 'O'), ('of', 'O'), ('the', 'O'), ('nineteenth', 'O'), ('century', 'O'), ('.', 'O')]
[('th

In [5]:
SEQ_LEN = 25

# Create word_to_idx and tag_to_idx mappings
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
tag_to_idx = {"<PAD>": 0}


def preprocess(dataset):
    # Extract sentences and tags
    sent = [[token.lower() for token, tag in sentence] for sentence in dataset]
    tags = [[tag for token, tag in sentence] for sentence in dataset]

    for i in range(len(sent)):
        while len(sent[i]) < SEQ_LEN:
            sent[i].append('<PAD>')
            tags[i].append('<PAD>')

        if len(sent[i]) > SEQ_LEN:
            sent[i] = sent[i][:SEQ_LEN]
            tags[i] = tags[i][:SEQ_LEN]
    
    for sentence_tags in tags:
        for tag in sentence_tags:
            if tag not in tag_to_idx:
                tag_to_idx[tag] = len(tag_to_idx)
    
    for sentence in sent:
        for word in sentence:
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)

    # Convert words and tags to indices
    X = torch.tensor([[word_to_idx.get(word, 1) for word in sentence] for sentence in sent], dtype=torch.int).type(torch.LongTensor)
    Y = torch.tensor([[tag_to_idx[tag] for tag in sentence] for sentence in tags], dtype=torch.int).type(torch.LongTensor)
    
    return X, Y


In [6]:
train_X, train_Y = preprocess(train_dataset)
dev_X, dev_Y = preprocess(dev_dataset)
test_X, test_Y = preprocess(test_dataset)

In [7]:
# Print the sizes of the datasets
print(f"Number of training examples: {len(train_X)}")
print(f"Number of validation examples: {len(dev_X)}")
print(f"Number of testing examples: {len(test_X)}")

Number of training examples: 16778
Number of validation examples: 871
Number of testing examples: 249980


In [8]:
%pip install pytorch-lightning 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.1 MB/s eta 0:00:00


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class NERModelFine(pl.LightningModule):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, num_layers=1, bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim) #B * seq_len, B * seq_len * embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=num_layers, bidirectional=bidirectional)
        #B * seq_len * embedding_dim -> B * seq_len * hidden_dim 
        #tags
        if bidirectional:
            self.fc = nn.Linear(2*hidden_dim, tagset_size)
        else:
            self.fc = nn.Linear(hidden_dim, tagset_size)
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, x):
        embeds = self.embedding(x)
        #print(embeds.shape)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.fc(lstm_out)
        tag_scores = nn.functional.log_softmax(tag_space, dim=2)
        return tag_scores
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer

In [10]:
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

EMBEDDING_DIM = 100
HIDDEN_DIM    = 100
NUM_EPOCHS    = 10
BATCH_SIZE    = 32

train_dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(dev_X, dev_Y)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [11]:
model = NERModelFine(vocab_size=len(word_to_idx), tagset_size=len(tag_to_idx), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, bidirectional=True)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")


In [12]:
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, callbacks=[early_stopping])

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

trainer.test(dataloaders=test_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 24.2 M
1 | lstm      | LSTM             | 161 K 
2 | fc        | Linear           | 2.8 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
24.3 M    Trainable params
0         Non-trainable params
24.3 M    Total params
97.351    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=6-step=3675.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=6-step=3675.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.3169195353984833     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3169195353984833}]

In [13]:
# define idx_to_tag
idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

In [14]:
from sklearn.metrics import classification_report



# define device
device = torch.device('cpu')

# Create a dataloader for the test set
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Set the model to evaluation mode
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

print(classification_report(y_true, y_pred))

                 precision    recall  f1-score   support

          <PAD>       1.00      1.00      1.00   2499089
B-CreativeWorks       0.46      0.35      0.40     62057
        B-Group       0.48      0.38      0.42     59928
     B-Location       0.59      0.56      0.58     67749
      B-Medical       0.31      0.26      0.28     22439
       B-Person       0.72      0.73      0.72    137485
      B-Product       0.23      0.18      0.20     27451
I-CreativeWorks       0.60      0.47      0.53    107119
        I-Group       0.58      0.49      0.53     73919
     I-Location       0.67      0.65      0.66     62771
      I-Medical       0.37      0.30      0.33     10580
       I-Person       0.74      0.74      0.74    153359
      I-Product       0.23      0.16      0.19     17415
              O       0.93      0.96      0.94   2948139

       accuracy                           0.92   6249500
      macro avg       0.57      0.52      0.54   6249500
   weighted avg       0.92   

In [15]:
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

In [16]:
# Set the model to evaluation mode
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)

        # Get back the sentence
        x_sent = [idx_to_word[i] for i in x.cpu().numpy().flatten().tolist()]

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]
        print("Sentence")
        print(x_sent)
        print("Predicted tags")
        print(y_pred)
        break

Sentence
['the', 'species', 'was', 'described', 'by', 'dietrich', 'brandis', 'after', 'the', 'forester', 't.', 'f.', 'bourdillon', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'by', 'this', 'time', 'she', 'was', 'competing', 'against', 'a', 'new', 'generation', 'of', 'young', 'drivers', 'including', 'stirling', 'moss', 'and', 'peter', 'collins', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'their', 'son', 'was', 'the', 'opera', 'producer', 'knut', 'hendriksen', '(', '1944', '–', '2020', ')', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'he', 'is', 'the', 'younger', 'brother', 'of', 'adam', 'mosseri', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'smes', ':', 'laura', 'j.', 'van', "'t", 'veer', 'et', 'al', '.', '(', 'nl', ')', 'for', 'their', 'gene', 'based', 'brea

In [17]:
save_dict = {'model_state_dict': model.state_dict()}
torch.save(save_dict, '/content/drive/MyDrive/Colab Notebooks/DL_ASS2/english_fine.pt')

In [19]:
# Load the saved model
import torch
load_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/DL_ASS2/english_fine.pt')

# Create a new model and optimizer
model_english = NERModelFine(vocab_size=len(word_to_idx), tagset_size=len(tag_to_idx), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, bidirectional=True)
# Load the state dictionary into the model and optimizer
model.load_state_dict(load_dict['model_state_dict'])

<All keys matched successfully>

In [20]:
import torch

def predict_sentence(sentence, model, word_to_idx, idx_to_tag, unknown_token="<UNK>"):
    # Tokenize the sentence
    words = sentence.split()

    # Convert words to lowercase
    words = [word.lower() for word in words]

    # Get the index of each word
    word_indices = [word_to_idx.get(word, word_to_idx[unknown_token]) for word in words]

    # Convert the word indices to a tensor
    tensor = torch.tensor(word_indices).unsqueeze(0)

    # Make a prediction with the model
    predictions = model(tensor)

    # Convert the predictions to tags
    tags = [idx_to_tag[pred] for pred in predictions.argmax(-1).cpu().numpy().flatten().tolist()]

    # Merge consecutive words with the same tag
    tagged_words = []
    for word, tag in zip(words, tags):
        if tag.startswith("B-"):
            tagged_words.append(f"{word}[{tag[2:]}]")
        elif tag.startswith("I-"):
            tagged_words[-1] += f" {word}"
        else:
            tagged_words.append(word)

    # Join the tagged words into a sentence
    tagged_sentence = " ".join(tagged_words)

    return tagged_sentence


print(predict_sentence("Jim bought 300 shares of Acme Corp. in 2022.", model, word_to_idx, idx_to_tag))

jim bought 300 shares of acme corp. in 2022.[Location]
